In [219]:
import pandas as pd
import math
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import datetime

### Relevant Data Set can be found on : https://data.world/raghav333/cricket-players-espn

## Reading Data

#### PySpark

In [220]:
spark=SparkSession.builder.appName('temp').getOrCreate()

#### Pandas

In [221]:
main=pd.read_csv("cricket_data.csv")

E:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (9,16,20,21,22,24,25,26,27,28,29,30,31,32,45,46,59,60,73,74,87,88,90,91,92,95,96,97,98,99,100,101,102) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Converting Pandas Dtype Object to String Dtype in order to use the same dateframe as Spark DataFrame

In [222]:
cols_dtypes=main.dtypes.values.tolist()
columns=main.columns.values.tolist()
for index,_ in enumerate(cols_dtypes):
    if str(cols_dtypes[index])=="object":
        main[columns[index]]=main[columns[index]].astype(str)

In [223]:
main_spark=spark.createDataFrame(main.sample(frac=0.01))

In [224]:
main_spark.printSchema()

root
 |-- ID: long (nullable = true)
 |-- NAME: string (nullable = true)
 |-- COUNTRY: string (nullable = true)
 |-- Full name: string (nullable = true)
 |-- Born: string (nullable = true)
 |-- Died: string (nullable = true)
 |-- Current age: string (nullable = true)
 |-- Major teams: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- Height: string (nullable = true)
 |-- Nickname: double (nullable = true)
 |-- Playing role: string (nullable = true)
 |-- Batting style: string (nullable = true)
 |-- Bowling style: string (nullable = true)
 |-- Other: string (nullable = true)
 |-- Relation: string (nullable = true)
 |-- In a nutshell: string (nullable = true)
 |-- DESCRIPTION: string (nullable = true)
 |-- AWARDS: string (nullable = true)
 |-- BATTING_Tests_Mat: double (nullable = true)
 |-- BATTING_Tests_Inns: string (nullable = true)
 |-- BATTING_Tests_NO: string (nullable = true)
 |-- BATTING_Tests_Runs: string (nullable = true)
 |-- BATTING_Tests_HS: string (nulla

In [58]:
main.head()

,ID,NAME,COUNTRY,Full name,Born,Died,Current age,Major teams,Education,Height,...,BOWLING_T20s_Runs,BOWLING_T20s_Wkts,BOWLING_T20s_BBI,BOWLING_T20s_BBM,BOWLING_T20s_Ave,BOWLING_T20s_Econ,BOWLING_T20s_SR,BOWLING_T20s_4w,BOWLING_T20s_5w,BOWLING_T20s_10
0,8772,Henry Arkell,England,Henry John Denham Arkell,"\nJune 26, 1898, Edmonton, Middlesex","March 12, 1982, Oxford (aged 83 years 259 days)",nan,Northamptonshire,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,532565,Richard Nyren,England,Richard Nyren,"\nApril 25, 1734, Eartham, Sussex","April 25, 1797, Lee or Leigh, Kent (aged 63 ye...",nan,Hampshire XI,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,16856,Sydney Maartensz,England,Sydney Gratien Adair Maartensz,"\nApril 14, 1882, Colombo, Ceylon","September 10, 1967, Pyrford, Woking, Surrey (a...",nan,Hampshire,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,16715,Brian Lander,England,Brian Richard Lander,"\nJanuary 9, 1942, Bishop Auckland, Co Durham",nan,77 years 73 days,"['Durham,', 'Minor Counties']",nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,15989,Derek Kenderdine,England,Derek Charles Kenderdine,"\nOctober 28, 1897, Chislehurst, Kent","August 28, 1947, Cambridge (aged 49 years 304 ...",nan,Royal Navy,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [59]:
main.columns.tolist()

['ID',
 'NAME',
 'COUNTRY',
 'Full name',
 'Born',
 'Died',
 'Current age',
 'Major teams',
 'Education',
 'Height',
 'Nickname',
 'Playing role',
 'Batting style',
 'Bowling style',
 'Other',
 'Relation',
 'In a nutshell',
 'DESCRIPTION',
 'AWARDS',
 'BATTING_Tests_Mat',
 'BATTING_Tests_Inns',
 'BATTING_Tests_NO',
 'BATTING_Tests_Runs',
 'BATTING_Tests_HS',
 'BATTING_Tests_Ave',
 'BATTING_Tests_BF',
 'BATTING_Tests_SR',
 'BATTING_Tests_100',
 'BATTING_Tests_50',
 'BATTING_Tests_4s',
 'BATTING_Tests_6s',
 'BATTING_Tests_Ct',
 'BATTING_Tests_St',
 'BATTING_ODIs_Mat',
 'BATTING_ODIs_Inns',
 'BATTING_ODIs_NO',
 'BATTING_ODIs_Runs',
 'BATTING_ODIs_HS',
 'BATTING_ODIs_Ave',
 'BATTING_ODIs_BF',
 'BATTING_ODIs_SR',
 'BATTING_ODIs_100',
 'BATTING_ODIs_50',
 'BATTING_ODIs_4s',
 'BATTING_ODIs_6s',
 'BATTING_ODIs_Ct',
 'BATTING_ODIs_St',
 'BATTING_T20Is_Mat',
 'BATTING_T20Is_Inns',
 'BATTING_T20Is_NO',
 'BATTING_T20Is_Runs',
 'BATTING_T20Is_HS',
 'BATTING_T20Is_Ave',
 'BATTING_T20Is_BF',
 'BATTIN

In [60]:
main['In a nutshell'].isnull().sum()

0

## Dropping those columns which have a high propotion of NULL Values

#### Pyspark

In [61]:
null_instances=main.isnull().sum()

In [62]:
null_instances_spark=main_spark.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in main_spark.columns])

In [63]:
null_instances_spark.show(1)

+---+----+-------+---------+----+-----+-----------+-----------+---------+------+--------+------------+-------------+-------------+-----+--------+-------------+-----------+------+-----------------+------------------+----------------+------------------+----------------+-----------------+----------------+----------------+-----------------+----------------+----------------+----------------+----------------+----------------+----------------+-----------------+---------------+-----------------+---------------+----------------+---------------+---------------+----------------+---------------+---------------+---------------+---------------+---------------+-----------------+------------------+----------------+------------------+----------------+-----------------+----------------+----------------+-----------------+----------------+----------------+----------------+----------------+----------------+-----------------------+------------------------+----------------------+------------------------+----

In [64]:
null_instances_spark_rdd=null_instances_spark.rdd

In [65]:
null_instances_spark_rdd.collect()

[Row(ID=0, NAME=1, COUNTRY=0, Full name=1, Born=0, Died=18328, Current age=10494, Major teams=1542, Education=20979, Height=22099, Nickname=22577, Playing role=21672, Batting style=8238, Bowling style=11551, Other=20971, Relation=21432, In a nutshell=22565, DESCRIPTION=0, AWARDS=0, BATTING_Tests_Mat=21689, BATTING_Tests_Inns=21689, BATTING_Tests_NO=21689, BATTING_Tests_Runs=21689, BATTING_Tests_HS=21689, BATTING_Tests_Ave=21689, BATTING_Tests_BF=22175, BATTING_Tests_SR=22175, BATTING_Tests_100=21689, BATTING_Tests_50=21689, BATTING_Tests_4s=22081, BATTING_Tests_6s=21800, BATTING_Tests_Ct=21689, BATTING_Tests_St=21689, BATTING_ODIs_Mat=21659, BATTING_ODIs_Inns=21659, BATTING_ODIs_NO=21659, BATTING_ODIs_Runs=21659, BATTING_ODIs_HS=21659, BATTING_ODIs_Ave=21659, BATTING_ODIs_BF=21801, BATTING_ODIs_SR=21801, BATTING_ODIs_100=21659, BATTING_ODIs_50=21659, BATTING_ODIs_4s=21852, BATTING_ODIs_6s=21846, BATTING_ODIs_Ct=21659, BATTING_ODIs_St=21659, BATTING_T20Is_Mat=22047, BATTING_T20Is_Inns=2

In [66]:
def col_func(x):
    list_=[]
    for col in list_columns:
        propotion_null=x[col]/count_rows
        if propotion_null<0.9:
            list_.append(col)
    return list_
        

In [67]:
list_columns=main_spark.columns
count_rows=main_spark.count()

In [68]:
count_rows

22577

In [69]:
columns_needed=null_instances_spark_rdd.map(col_func).collect()

In [70]:
null_instances=null_instances.map(lambda x:x/main.shape[0])

In [71]:
null_instances[null_instances>0.90].index

Index(['Nickname', 'BATTING_Tests_Mat', 'BATTING_ODIs_Mat',
       'BATTING_T20Is_Mat', 'BATTING_T20s_Mat', 'BOWLING_Tests_Mat',
       'BOWLING_ODIs_Mat', 'BOWLING_T20Is_Mat', 'BOWLING_T20s_Mat'],
      dtype='object')

In [72]:
main_spark=main_spark.drop('Education', 'Height', 'Nickname', 'Playing role', 'Other', 'Relation',
       'In a nutshell')

#### Pandas

In [73]:
main.drop(['Education', 'Height', 'Nickname', 'Playing role', 'Other', 'Relation',
       'In a nutshell'],1,inplace=True)

In [74]:
main["Born"]=main["Born"].str.replace("\n","")

## Forming the Date Birth Column which have Year+Date the player was born

#### Pyspark

In [66]:
split_col = split(main_spark['Born'], ',')

In [67]:
main_spark = main_spark.withColumn('NAME1', split_col.getItem(0))
main_spark = main_spark.withColumn('NAME2', split_col.getItem(1))

In [68]:
main_spark.select("NAME1","NAME2").show()

+---------------+--------+
|          NAME1|   NAME2|
+---------------+--------+
|      \nJune 26|    1898|
|     \nApril 25|    1734|
|     \nApril 14|    1882|
|    \nJanuary 9|    1942|
|   \nOctober 28|    1897|
|     \nMarch 24|    1979|
|      \nJuly 17|   1999 |
|   \nDecember 6|    1993|
|  \nFebruary 26|    1929|
|   \nJanuary 21|   2003 |
|\ndate unknown |    null|
|\ndate unknown |    null|
|     \nApril 23|    1986|
|    \nJanuary 8|    1999|
|\ndate unknown |    null|
|\ndate unknown |    null|
|     \nMarch 27|    1991|
|    \nOctober 1|    1987|
|         \n1947| Mombasa|
|        \n1975 |    null|
+---------------+--------+
only showing top 20 rows



In [69]:
main_spark=main_spark.na.fill('',["NAME2"])

In [70]:
def fill_Date_birth_spark(x):
    name_2=x["NAME2"]
    name_1=x['NAME1']
    
    name_2=name_2.strip()
    name_1=name_1.strip()
    
    if not name_2.isnumeric():
        return name_1
    else:
        return name_1+"-"+name_2
   

In [71]:
main_spark_rdd=main_spark.rdd

In [72]:
main_spark_rdd_names_join=main_spark_rdd.map(fill_Date_birth_spark)

In [73]:
list_rdd_born=main_spark_rdd_names_join.collect()

In [74]:
main_spark = main_spark.repartition(1).withColumn(
    "Date_Birth", 
    udf(lambda id: list_rdd_born[id])(monotonically_increasing_id()))

In [75]:
main_spark=main_spark.drop("NAME1","NAME2")

In [76]:
main_spark.select("Date_Birth").show()

+----------------+
|      Date_Birth|
+----------------+
|    June 26-1898|
|   April 25-1734|
|   April 14-1882|
|  January 9-1942|
| October 28-1897|
|   March 24-1979|
|    July 17-1999|
| December 6-1993|
|February 26-1929|
| January 21-2003|
|    date unknown|
|    date unknown|
|   April 23-1986|
|  January 8-1999|
|    date unknown|
|    date unknown|
|   March 27-1991|
|  October 1-1987|
|            1947|
|            1975|
+----------------+
only showing top 20 rows



#### Pandas

In [77]:
date_splitted=main["Born"].str.split(",",expand=True)
date_splitted.iloc[:,1]=date_splitted.iloc[:,1].fillna('')

In [5]:
def fill_date_birth(x):
    x[1]=x[1].strip()
    if not x[1].isnumeric():
        return x[0]
    else:
        return x[0]+"-"+x[1]

main["Date_Birth"]=date_splitted.apply(fill_date_birth,axis=1)

### What to do with players whose date is not known

#### PySpark

In [78]:
main_spark = main_spark.withColumn('Date_Birth', trim(col('Date_Birth')))

In [79]:
mode_date_birth_spark=main_spark.filter(~main_spark.Date_Birth.contains('unknown')).groupby('Date_Birth').count().orderBy('count',ascending=False).first()[0]

In [80]:
main_spark=main_spark.withColumn('Date_Birth', when(main_spark.Date_Birth.rlike('known|Known|circa|Details|Hong|Cheema|circ|Pakistan|Goa|Kamrup|date unknown'),mode_date_birth_spark).otherwise(
    main_spark['Date_Birth']
))


In [81]:
main_spark=main_spark.withColumn('Date_Birth', regexp_replace('Date_Birth', ' ', '/')) 
main_spark=main_spark.withColumn('Date_Birth', regexp_replace('Date_Birth', '-', '/')) 
main_spark.select('Date_Birth').show()

+----------------+
|      Date_Birth|
+----------------+
|    June/26/1898|
|   April/25/1734|
|   April/14/1882|
|  January/9/1942|
| October/28/1897|
|   March/24/1979|
|    July/17/1999|
| December/6/1993|
|February/26/1929|
| January/21/2003|
|February/24/1975|
|February/24/1975|
|   April/23/1986|
|  January/8/1999|
|February/24/1975|
|February/24/1975|
|   March/27/1991|
|  October/1/1987|
|            1947|
|            1975|
+----------------+
only showing top 20 rows



In [82]:
def date_birth_func_spark(x):
    month_to_numeric={
        "January":"01",
        "February":"02",
        "March":"03",
        "April":"04",
        "May":"05",
        "June":"06",
        "July":"07",
        "August":"08",
        "September":"09",
        "October":"10",
        "November":"11",
        "December":"12",
        "Q1":"1" 
    }
    
    date_instance=x.split("/")
    if '' in date_instance:
        date_instance[date_instance.index('')]='1994'
    if '0000' in date_instance:
        date_instance[date_instance.index('0000')]='1994'
    if len(date_instance)==1:
        return datetime.strptime("01/01/"+date_instance[0],'%m/%d/%Y')
    else:
        if date_instance[0] in month_to_numeric.keys():
            date_instance[0]=month_to_numeric[date_instance[0]]
        return datetime.strptime("/".join(date_instance),'%m/%d/%Y')

In [83]:
udf_function = udf(lambda x: date_birth_func_spark(x),DateType())
main_spark=main_spark.withColumn('Date_Birth',udf_function(col('Date_Birth')))

#### Pandas

In [6]:
main["Date_Birth"]=main["Date_Birth"].str.strip()

In [7]:
mode_date_birth=main.loc[~(main["Date_Birth"].str.contains('unknown')),'Date_Birth'].mode().values.tolist()[0]

In [8]:
main.loc[(main["Date_Birth"].str.contains('known|Known|circa|Details|Hong|Cheema|circ|Pakistan|Goa|Kamrup|Mansa|Matale|Badulla|Balapitiya|Kiribathgoda|Colombo|Westminster|Polonnaruwa')),'Date_Birth']="January 1-"+mode_date_birth

In [9]:
main["Date_Birth"]=main["Date_Birth"].str.replace(" ","/")
main["Date_Birth"]=main["Date_Birth"].str.replace("-","/")

In [10]:
final_list=[]
def date_birth_func(x):
    month_to_numeric={
        "January":"01",
        "February":"02",
        "March":"03",
        "April":"04",
        "May":"05",
        "June":"06",
        "July":"07",
        "August":"08",
        "September":"09",
        "October":"10",
        "November":"11",
        "December":"12",
        "Q1":"1" 
    }
    if x=='':
        return datetime.strptime("01/01/"+mode_date_birth, '%m/%d/%Y')
    date_instance=x.split("/")
    try:
        if '' in date_instance:
            date_instance[date_instance.index('')]=mode_date_birth
            return datetime.strptime("/".join(date_instance), '%m/%d/%Y')
        if '0000' in date_instance:
            date_instance[date_instance.index('0000')]=mode_date_birth
        if len(date_instance)==1:
            return datetime.strptime("01/01/"+date_instance[0], '%m/%d/%Y')
        elif len(date_instance)==2:
            date_instance[0]=month_to_numeric[date_instance[0]]
            return datetime.strptime(date_instance[0]+"/01/"+date_instance[1], '%m/%d/%Y')
        else:
            if date_instance[0] in month_to_numeric.keys():
                date_instance[0]=month_to_numeric[date_instance[0]]
            return datetime.strptime("/".join(date_instance), '%m/%d/%Y')
    except:
        final_list.append(x)
   
        
    
    
   
    
    

In [11]:
main["Born"]=main["Date_Birth"].map(date_birth_func)

In [12]:
main.drop('Date_Birth',axis=1,inplace=True)

## Forming the Date Death Column which have Year+Date the player was born

#### Pyspark

In [84]:
split_col = split(main_spark['Died'], ',')

In [85]:
main_spark = main_spark.withColumn('NAME1', split_col.getItem(0))
main_spark = main_spark.withColumn('NAME2', split_col.getItem(1))

In [86]:
main_spark=main_spark.fillna('',subset=['NAME2'])

In [87]:
custom_func_name1_name2=udf(lambda x,y:x+"-"+y)

In [88]:
main_spark=main_spark.withColumn('Died',custom_func_name1_name2(col('NAME1'),col('NAME2')))

In [89]:
main_spark=main_spark.drop('NAME1','NAME2')

In [90]:
mode_date_died_spark=main_spark.filter(~main_spark.Died.rlike('nan|unknown|Presumed Dead|Known|Unknown')).groupby('Died').count().orderBy('count',ascending=False).first()[0]

In [91]:
mode_date_died_spark

'September 10- 1967'

In [92]:
global mode_year_spark
mode_year_spark=mode_date_died_spark.split('- ')[-1]

In [93]:
main_spark=main_spark.withColumn('Died', when(main_spark.Died.rlike('unknown|Presumed Dead|Known|Unknown|0000|aged|Duncton|Leicester|Tasmania'),mode_date_died_spark).otherwise(
    main_spark['Died']
))

In [94]:
main_spark.select('Died').show()

+------------------+
|              Died|
+------------------+
|    March 12- 1982|
|    April 25- 1797|
|September 10- 1967|
|              nan-|
|   August 28- 1947|
|              nan-|
|              nan-|
|              nan-|
|              nan-|
|              nan-|
|              nan-|
|              nan-|
|              nan-|
|              nan-|
|              nan-|
|              nan-|
|              nan-|
|              nan-|
|              nan-|
|              nan-|
+------------------+
only showing top 20 rows



In [95]:
main_spark=main_spark.withColumn('Died', regexp_replace('Died', '- ', '/')) 
main_spark=main_spark.withColumn('Died', regexp_replace('Died', ' ', '/')) 
main_spark=main_spark.withColumn('Died', regexp_replace('Died', 'nan/', 'nan')) 


In [96]:
def date_death_func_spark(x):
    month_to_numeric={
        "January":"01",
        "February":"02",
        "March":"03",
        "April":"04",
        "May":"05",
        "June":"06",
        "July":"07",
        "August":"08",
        "September":"09",
        "October":"10",
        "November":"11",
        "December":"12",
        "Q1":"1" ,
        'Q2':'4'
    }  
   
    if x=='nan':
        return 'nan'
    date_list=x.split('/')[:3]
    all_numeric=True
    index_to_pop=[]
    for index,instance in enumerate(date_list):
        if not instance.isnumeric() and instance not in month_to_numeric.keys():
            all_numeric=False
        else:    
            index_to_pop.append(index)
    if all_numeric:
        date_list[0]=month_to_numeric[date_list[0]]
        return datetime.strptime('/'.join(date_list),'%m/%d/%Y')
        
    else:

        new_date_list=[]
        for index in index_to_pop:
            new_date_list.append(date_list[index])
        if len(new_date_list)==1:
            return datetime.strptime('01/'+'01/'+new_date_list[0],'%m/%d/%Y')
        elif len(new_date_list)==2 and new_date_list[0] in month_to_numeric.keys():
            new_date_list[0]=month_to_numeric[new_date_list[0]]
            return datetime.strptime(new_date_list[0]+'/01/'+new_date_list[1],'%m/%d/%Y')
        else:
            return datetime.strptime('01/'+'01/'+mode_year_spark,'%m/%d/%Y')
                
    
        
            
    
      

In [97]:
death_udf=udf(lambda x:date_death_func_spark(x),DateType())
main_spark=main_spark.withColumn('Died',death_udf(col('Died')))

#### Pandas

In [13]:
main['Died']=main['Died'].map(lambda x:"-".join(x.split(',')[:2]))

In [14]:
mode_died=main.loc[~(main['Died'].str.contains('nan|unknown|Presumed Dead|Known|Unknown')),'Died'].mode().values.tolist()[0]

In [15]:
mode_year=mode_died.split('- ')[-1]

In [16]:
main.loc[(main['Died'].str.contains('unknown|Presumed Dead|Known|Unknown|0000|aged|Duncton|Leicester|Tasmania')),'Died']=mode_died

In [17]:
main['Died']=main['Died'].str.replace('- ','/').str.replace(' ','/')

In [18]:
def date_death_func(x):
    month_to_numeric={
        "January":"01",
        "February":"02",
        "March":"03",
        "April":"04",
        "May":"05",
        "June":"06",
        "July":"07",
        "August":"08",
        "September":"09",
        "October":"10",
        "November":"11",
        "December":"12",
        "Q1":"1" ,
        'Q2':'4'
    }  
   
    if x=='nan':
        return 'nan'
    date_list=x.split('/')[:3]
    all_numeric=True
    index_to_pop=[]
    for index,instance in enumerate(date_list):
        if not instance.isnumeric() and instance not in month_to_numeric.keys():
            all_numeric=False
        else:    
            index_to_pop.append(index)
    if all_numeric:
        date_list[0]=month_to_numeric[date_list[0]]
        return datetime.strptime('/'.join(date_list), '%m/%d/%Y')
        
    else:

        new_date_list=[]
        for index in index_to_pop:
            new_date_list.append(date_list[index])
        if len(new_date_list)==1:
            return datetime.strptime('01/'+'01/'+new_date_list[0], '%m/%d/%Y')
        elif len(new_date_list)==2 and new_date_list[0] in month_to_numeric.keys():
            new_date_list[0]=month_to_numeric[new_date_list[0]]
            return datetime.strptime(new_date_list[0]+'/01/'+new_date_list[1], '%m/%d/%Y')
        else:
            return datetime.strptime('01/'+'01/'+mode_year, '%m/%d/%Y')
                
    
        
            
    
      

In [19]:
main['Died']=main['Died'].map(date_death_func)

## Forming the Age Column
    -  Age=Death-Birth (IF died)
    -  Age=Today()- Birth (IF Alive)

#### PySpark

In [98]:
main_spark=main_spark.withColumn('Age',col('Current age'))

In [99]:
main_spark=main_spark.withColumnRenamed("Current age","Current_age")

In [100]:
case_1_func=udf(lambda x:(datetime.now()-x).days/365)

In [101]:
main_spark=main_spark.withColumn("Age",when(~main_spark.Age.contains('nan'),case_1_func(col('Date_Birth'))).otherwise(main_spark['Age']))

In [102]:
Case_2_func=udf(lambda x,y :(x-y).days/365)

In [103]:
main_spark=main_spark.withColumn("Age",when(main_spark.Age.contains('nan') & ~main_spark.Died.isNull() ,Case_2_func(col('Died'),col('Date_Birth'))).otherwise(main_spark['Age']))

In [104]:
main_spark.select('Age').show()

+------------------+
|               Age|
+------------------+
| 83.76438356164384|
| 63.04383561643836|
| 85.46301369863014|
| 80.22739726027397|
|49.863013698630134|
|              43.0|
|22.671232876712327|
|28.284931506849315|
|  93.1041095890411|
|19.153424657534245|
|-8.153424657534247|
|-8.153424657534247|
|35.912328767123284|
| 23.19178082191781|
|-8.153424657534247|
|-8.153424657534247|
|30.983561643835618|
| 34.47123287671233|
| 75.24657534246575|
|47.227397260273975|
+------------------+
only showing top 20 rows



In [105]:
main_spark=main_spark.withColumn('Age',when(main_spark.Current_age.contains('nan') & main_spark.Died.isNull(),case_1_func(col('Date_Birth'))).otherwise(main_spark['Age']))

In [106]:
main_spark=main_spark.withColumn('Died',when(main_spark.Age<0,None).otherwise(main_spark.Died))

In [115]:
main_spark=main_spark.withColumn('Age',main_spark.Age.cast(IntegerType()))

In [131]:
main_spark=main_spark.withColumn('Age',when((main_spark.Age<0) & (main_spark.Died.isNull()),case_1_func(col('Date_Birth'))).otherwise(main_spark['Age']))

In [132]:
main_spark=main_spark.withColumn('Age',when(main_spark.Age>150,150).otherwise(main_spark.Age))

In [133]:
main_spark=main_spark.drop('Current_age')

In [137]:
main_spark=main_spark.withColumn('Age',floor(ceil('Age')))

#### Pandas

In [37]:
main['Age']=main['Current age']

In [38]:
main.loc[(main['Current age']!='nan'),'Age']=main.loc[(main['Current age']!='nan'),:].apply(lambda x:(datetime.today()-x['Born']).days/365,axis=1)#### Case-1 Those who are still alive, what is their updated age?

In [40]:
main.loc[(main['Current age']=='nan') & ~(main['Died'].isnull()) ,'Age']=main.loc[(main['Current age']=='nan') & ~(main['Died'].isnull()) ,:].apply(lambda x:(x['Died']-x['Born']).days/365,axis=1)### Case-2 Those who have died and their dead date is known

In [41]:
main.loc[(main['Current age']=='nan') & (main['Died'].isnull()) ,'Age']=main.loc[(main['Current age']=='nan') & (main['Died'].isnull()) ,:].apply(lambda x:(datetime.today()-x['Born']).days/365,axis=1)## Case Those who are alive but their current age is nan

In [49]:
main.loc[(main['Age']<0),'Died']=None #### This is happening because some of died values were imputed from the mode which might have the cause the died column to be less than Birth therefore making them null makes sense

In [52]:
main.loc[(main['Age']<0)&(main['Died']).isnull(),'Age']=main.loc[(main['Age']<0)&(main['Died']).isnull(),:].apply(lambda x:(datetime.today()-x['Born']).days/365,axis=1)##### Those whose age is negative due to death mode imputation--assumption is that they are still alive.

In [57]:
main.loc[(main['Age']>150),'Age']=150 ### No one can live that long :D 

In [ ]:
main['Age']=main['Age'].map(lambda x:math.ceil(x))

In [60]:
main.drop('Current age',axis=1,inplace=True)

## Forming the Country Column

#### PySpark

In [11]:
main_spark=main_spark.withColumnRenamed('COUNTRY','Country')

In [13]:
main_spark=main_spark.withColumn('Country',trim(col('Country')))

In [15]:
main_spark.select('Country').show()

+--------------------+
|             Country|
+--------------------+
|            Pakistan|
|             England|
|           Australia|
|               India|
|           Australia|
|               India|
|               India|
|            Zimbabwe|
|            Pakistan|
|        South Africa|
|             England|
|        South Africa|
|               India|
|               India|
|               India|
|            Zimbabwe|
|United Arab Emirates|
|           Sri Lanka|
|             England|
|               India|
+--------------------+
only showing top 20 rows



#### Pandas

In [9]:
main=main.rename(columns={'COUNTRY':'Country'})

In [10]:
main['Country']=main['Country'].str.strip()

## Forming the Major Team Column

#### Pandas

In [18]:
main=main.rename(columns={'Major teams':'MajorTeam'})

In [33]:
def team_cleaning(x):
    if '[' in x:
        first_team=x.split(',')[0]
        first_team=first_team.replace('[','')
        first_team=first_team.replace("'",'')
        return first_team
    return x

In [40]:
main['MajorTeam']=main['MajorTeam'].map(team_cleaning).str.strip().astype(str)

#### PySpark

In [42]:
main_spark=main_spark.withColumnRenamed('Major teams','MajorTeam')

In [45]:
main_spark=main_spark.withColumn('MajorTeam',udf(team_cleaning)(col('MajorTeam')))

## Forming the Major Batting style Column

#### PySpark

In [38]:
main_spark=main_spark.withColumnRenamed('Batting style','Batting_style')

In [41]:
main_spark=main_spark.withColumn('Batting_style',when(main_spark.Batting_style.rlike('Right-hand bat|nan'),'Right-hand bat').otherwise(main_spark.Batting_style))

In [51]:
main_spark.filter(main_spark.Batting_style.isNull()).count()

0

#### Pandas

In [37]:
main['Batting style'].value_counts()

Right-hand bat    80681
Left-hand bat      9627
Name: Batting style, dtype: int64

In [36]:
main.loc[(main['Batting style'].str.contains('Right-hand bat, Right-hand bat|nan')),"Batting style"]='Right-hand bat'

## Forming the Major Bowling style Column

#### PySpark

In [26]:
main_spark=main_spark.withColumnRenamed('Bowling style','Bowling_style')

In [27]:
main_spark=main_spark.withColumn('Bowling_style',when(main_spark.Bowling_style.rlike('nan'),'Unknown Style').otherwise(main_spark.Bowling_style))

In [28]:
main_spark.select('Bowling_style').show()

+--------------------+
|       Bowling_style|
+--------------------+
|       Unknown Style|
|       Unknown Style|
|       Unknown Style|
|       Unknown Style|
|       Unknown Style|
|       Unknown Style|
|    Right-arm medium|
|       Unknown Style|
|       Unknown Style|
|  Right-arm offbreak|
|    Right-arm medium|
|    Right-arm medium|
|       Unknown Style|
|       Unknown Style|
|    Right-arm medium|
|       Unknown Style|
|            Legbreak|
|       Unknown Style|
|       Unknown Style|
|Right-arm fast-me...|
+--------------------+
only showing top 20 rows



#### Pandas

In [17]:
main.loc[(main['Bowling style'].str.contains('nan')),'Bowling style']='Unknown Style'

## Forming the Major BATTING Run Column

#### PySpark

In [225]:
dtypes_cols=[x[1] for x in main_spark.dtypes]

In [226]:
for col_1 in ['BATTING_Tests','BATTING_ODIs','BATTING_T20Is']:
    for col_2 in ['_Mat','_Inns','_NO','_Runs','_HS','_Ave','_BF','_SR','_100','_50','_4s','_6s','_Ct','_St']:
        dtype_=str(dtypes_cols[main_spark.columns.index(col_1+col_2)])
        main_spark=main_spark.fillna(0,subset=col_1+col_2)
        if not 'double' in dtype_ or not 'bigint' in dtype_:
            
            main_spark=main_spark.withColumn(col_1+col_2,udf(lambda x:str(x))(col(col_1+col_2)))
            main_spark=main_spark.withColumn(col_1+col_2,main_spark[col_1+col_2].cast('string'))
            main_spark=main_spark.replace('nan','-1',subset=[col_1+col_2])
            main_spark=main_spark.replace('*','-1',subset=[col_1+col_2])
            main_spark=main_spark.replace('-','-1',subset=[col_1+col_2])

        main_spark=main_spark.withColumn(col_1+col_2,main_spark[col_1+col_2].cast('double'))

In [227]:
main_spark.show()

+-------+-----------------+--------------------+--------------------+--------------------+----+-----------------+--------------------+--------------------+------+--------+------------+--------------+--------------------+------+--------+-------------+--------------------+--------------------+-----------------+------------------+----------------+------------------+----------------+-----------------+----------------+----------------+-----------------+----------------+----------------+----------------+----------------+----------------+----------------+-----------------+---------------+-----------------+---------------+----------------+---------------+---------------+----------------+---------------+---------------+---------------+---------------+---------------+-----------------+------------------+----------------+------------------+----------------+-----------------+----------------+----------------+-----------------+----------------+----------------+----------------+----------------+----

#### Pandas

In [126]:
for col_1 in ['BATTING_Tests','BATTING_ODIs','BATTING_T20Is','BATTING_First-class','BATTING_List A','BATTING_T20s']:
    for col_2 in ['_Mat','_Inns','_NO','_Runs','_HS','_Ave','_BF','_SR','_100','_50','_4s','_6s','_Ct','_St']:
        dtype_=str(main.dtypes.values.tolist()[main.columns.values.tolist().index(col_1+col_2)])
        main[col_1+col_2]=main[col_1+col_2].fillna(0)
        if not 'int' in dtype_:
            
            main[col_1+col_2]=main[col_1+col_2].map(lambda x:str(x))
            main[col_1+col_2]=main[col_1+col_2].astype(str)
            main.loc[(main[col_1+col_2].str.contains('nan')),col_1+col_2]=-1
            main[col_1+col_2]=main[col_1+col_2].str.replace('*','')
            main[col_1+col_2]=main[col_1+col_2].str.replace('-','-1')
        main[col_1+col_2]=main[col_1+col_2].astype(float)


BATTING_Tests_Mat
BATTING_Tests_Inns
BATTING_Tests_NO
BATTING_Tests_Runs
BATTING_Tests_HS
BATTING_Tests_Ave
BATTING_Tests_BF
BATTING_Tests_SR
BATTING_Tests_100
BATTING_Tests_50
BATTING_Tests_4s
BATTING_Tests_6s
BATTING_Tests_Ct
BATTING_Tests_St
BATTING_ODIs_Mat
BATTING_ODIs_Inns
BATTING_ODIs_NO
BATTING_ODIs_Runs
BATTING_ODIs_HS
BATTING_ODIs_Ave
BATTING_ODIs_BF
BATTING_ODIs_SR
BATTING_ODIs_100
BATTING_ODIs_50
BATTING_ODIs_4s
BATTING_ODIs_6s
BATTING_ODIs_Ct
BATTING_ODIs_St
BATTING_T20Is_Mat
BATTING_T20Is_Inns
BATTING_T20Is_NO
BATTING_T20Is_Runs
BATTING_T20Is_HS
BATTING_T20Is_Ave
BATTING_T20Is_BF
BATTING_T20Is_SR
BATTING_T20Is_100
BATTING_T20Is_50
BATTING_T20Is_4s
BATTING_T20Is_6s
BATTING_T20Is_Ct
BATTING_T20Is_St
BATTING_First-class_Mat
BATTING_First-class_Inns
BATTING_First-class_NO
BATTING_First-class_Runs
BATTING_First-class_HS
BATTING_First-class_Ave
BATTING_First-class_BF
BATTING_First-class_SR
BATTING_First-class_100
BATTING_First-class_50
BATTING_First-class_4s
BATTING_First-class